In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [71]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [78]:
df = pd.read_csv(r'/content/gdrive/MyDrive/patent project/clean_data_with_enrich.csv',dtype={'filing_year':'str'})

<ipython-input-78-7ba450f1f037>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/content/gdrive/MyDrive/patent project/clean_data_with_enrich.csv',dtype={'filing_year':'str'})


In [73]:
df.head()

,application_number,uspc_class,one_if_male,one_if_female,filing_year,one_if_patented,one_if_abandoned,one_if_pending,one_if_small,Biotechnology,...,words_per_sentence,sentences_per_paragraph,type_token_ratio,syllables,sentences,long_words,complex_words,Gender_Target,Patent_Target,Parent_Catgeory
0,13054882,701,1,0,2013,1,0,0,0,0,...,27.250000,4.0,0.504587,183.0,4.0,35.0,28.0,Male,patented,Trans
1,13260099,424,1,0,2013,0,1,0,0,1,...,37.666667,3.0,0.530973,199.0,3.0,36.0,26.0,Male,abandoned,Biotechnology
2,13261421,424,1,0,2013,1,0,0,1,1,...,19.000000,2.0,0.736842,77.0,2.0,21.0,16.0,Male,patented,Biotechnology
3,13261486,136,1,0,2013,1,0,0,1,0,...,49.000000,3.0,0.653061,282.0,3.0,67.0,51.0,Male,patented,Chem_Material_Engr
4,13261494,362,1,0,2013,1,0,0,1,0,...,45.666667,3.0,0.583942,200.0,3.0,39.0,25.0,Male,patented,Semis


In [74]:
gender_df = df[['clean_abstract','Gender_Target']]

Check Top 20 words by Gender

In [75]:
def get_ngrams(data, n, g):
    vec = CountVectorizer(ngram_range=(g, g)).fit(data)
    bag_of_words = vec.transform(data) #sparse matrix of count_vectorizer
    sum_words = bag_of_words.sum(axis=0) #total number of words
    sum_words = np.array(sum_words)[0].tolist() #convert to list
    words_freq = [(word, sum_words[idx]) for word, idx in vec.vocabulary_.items()] #get word freqency for word location in count vec
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True) #key is used to perform sorting using word_freqency
    return words_freq[:n]

In [76]:
#female unigrams
female_unigrams = get_ngrams(gender_df[gender_df['Gender_Target']=='Female']['clean_abstract'],50,1)

KeyboardInterrupt: ignored

In [ ]:
male_unigrams = get_ngrams(gender_df[gender_df['Gender_Target']=='Male']['clean_abstract'],50,1)

In [ ]:
import operator

#only in female
for m_word in female_unigrams:
  if m_word[0] not in  list(map(operator.itemgetter(0), male_unigrams)):
    print(m_word[0])

In [ ]:
for m_word in male_unigrams:
  if m_word[0] not in  list(map(operator.itemgetter(0), female_unigrams)):
    print(m_word[0])

In [ ]:
#get bigrams
female_bigrams = get_ngrams(gender_df[gender_df['Gender_Target']=='Female']['clean_abstract'],50,2)
male_bigrams = get_ngrams(gender_df[gender_df['Gender_Target']=='Male']['clean_abstract'],50,2)

In [ ]:
for m_word in female_bigrams:
  if m_word[0] not in  list(map(operator.itemgetter(0), male_bigrams)):
    print(m_word[0])

In [ ]:
for m_word in male_bigrams:
  if m_word[0] not in  list(map(operator.itemgetter(0), female_bigrams)):
    print(m_word[0])

**TFIDF**

In [ ]:
gender_df['Gender_Target'].value_counts()

In [ ]:
gender_df['Gender_Target'] = gender_df['Gender_Target'].apply(lambda x: 1 if x =='Female' else 0)

In [ ]:
#lr classfier
word_dict = {}
for i in range(9):
  only_male = gender_df.loc[gender_df['Gender_Target']==0,:].sample(26200)
  only_female = gender_df.loc[gender_df['Gender_Target']==1,:]
  all_data = pd.concat([only_male,only_female],ignore_index=True)
  tfidf_vect = TfidfVectorizer(ngram_range=(1,2))
  train, test= train_test_split(all_data, test_size=0.2, random_state=42)
  Xtrain, ytrain = train['clean_abstract'], train['Gender_Target']
  Xtest, ytest = test['clean_abstract'], test['Gender_Target']
  Xtrain_tfidf = tfidf_vect.fit_transform(Xtrain)
  Xtest_tfidf = tfidf_vect.transform(Xtest)
  lr = LogisticRegression(max_iter=1000)
  vocab = tfidf_vect.vocabulary_
  feature_names = {v: k for k, v in vocab.items()}
  lr.fit(Xtrain_tfidf,ytrain)
  lr.fit(Xtrain_tfidf,ytrain)
  p1=lr.predict(Xtest_tfidf)
  s1=accuracy_score(ytest,p1)
  print("Logistic Regression Accuracy :", "{:.2f}%".format(100*s1))
  coef = lr.coef_[0]
  features = sorted([(feature_names[idx], coef[idx]) for idx in range(len(coef))], key=lambda x: x[1], reverse=True)
  for feature in features[:20]:
    print(feature)
    if feature[0] in word_dict:
      word_dict[feature[0]]+=1
    else:
      word_dict[feature[0]] = 1

In [ ]:
word_importence = sorted(word_dict.items(),key=lambda x: x[1],reverse=True)

In [ ]:
word_importence

In [ ]:
#only small
only_small = df[df['one_if_small']==1][['clean_abstract','Gender_Target']]

In [ ]:
only_small['Gender_Target'] = only_small['Gender_Target'].apply(lambda x: 1 if x =='Female' else 0)

In [ ]:
only_small['Gender_Target'].value_counts()

In [ ]:
#lr classfier
word_dict = {}
for i in range(7):
  only_male = only_small.loc[gender_df['Gender_Target']==0,:].sample(19000)
  only_female = only_small.loc[gender_df['Gender_Target']==1,:]
  all_data = pd.concat([only_male,only_female],ignore_index=True)
  tfidf_vect = TfidfVectorizer(ngram_range=(1,2))
  train, test= train_test_split(all_data, test_size=0.2, random_state=42)
  Xtrain, ytrain = train['clean_abstract'], train['Gender_Target']
  Xtest, ytest = test['clean_abstract'], test['Gender_Target']
  Xtrain_tfidf = tfidf_vect.fit_transform(Xtrain)
  Xtest_tfidf = tfidf_vect.transform(Xtest)
  lr = LogisticRegression(max_iter=1000)
  vocab = tfidf_vect.vocabulary_
  feature_names = {v: k for k, v in vocab.items()}
  lr.fit(Xtrain_tfidf,ytrain)
  lr.fit(Xtrain_tfidf,ytrain)
  p1=lr.predict(Xtest_tfidf)
  s1=accuracy_score(ytest,p1)
  print("Logistic Regression Accuracy :", "{:.2f}%".format(100*s1))
  coef = lr.coef_[0]
  features = sorted([(feature_names[idx], coef[idx]) for idx in range(len(coef))], key=lambda x: x[1], reverse=True)
  for feature in features[:20]:
    print(feature)
    if feature[0] in word_dict:
      word_dict[feature[0]]+=1
    else:
      word_dict[feature[0]] = 1

In [ ]:
word_importence = sorted(word_dict.items(),key=lambda x: x[1],reverse=True)

In [ ]:
word_importence

In [ ]:
#by category
cat_list = set(df['Parent_Catgeory'].to_list())

In [ ]:
cat_list

In [ ]:
import math
math.ceil(7.3)

In [ ]:
cat_dict = {}
for cat in cat_list:
  word_dict = {}
  cat_df = df[(df['one_if_small']==1) & (df['Parent_Catgeory']==cat)][['clean_abstract','Gender_Target']]
  cat_df['Gender_Target'] = cat_df['Gender_Target'].apply(lambda x: 1 if x =='Female' else 0)
  how_many_iter = math.ceil(cat_df[cat_df['Gender_Target']==0].shape[0]/cat_df[cat_df['Gender_Target']==1].shape[0])
  sample_size = cat_df[cat_df['Gender_Target']==1].shape[0]
  for i in range(how_many_iter):
    print(cat,i)
    only_male = cat_df.loc[cat_df['Gender_Target']==0,:].sample(sample_size)
    only_female = cat_df.loc[cat_df['Gender_Target']==1,:]
    all_data = pd.concat([only_male,only_female],ignore_index=True)
    tfidf_vect = TfidfVectorizer(ngram_range=(1,2))
    train, test= train_test_split(all_data, test_size=0.2, random_state=42)
    Xtrain, ytrain = train['clean_abstract'], train['Gender_Target']
    Xtest, ytest = test['clean_abstract'], test['Gender_Target']
    Xtrain_tfidf = tfidf_vect.fit_transform(Xtrain)
    Xtest_tfidf = tfidf_vect.transform(Xtest)
    lr = LogisticRegression(max_iter=1000)
    vocab = tfidf_vect.vocabulary_
    feature_names = {v: k for k, v in vocab.items()}
    lr.fit(Xtrain_tfidf,ytrain)
    lr.fit(Xtrain_tfidf,ytrain)
    p1=lr.predict(Xtest_tfidf)
    s1=accuracy_score(ytest,p1)
    print("Logistic Regression Accuracy :", "{:.2f}%".format(100*s1))
    coef = lr.coef_[0]
    features = sorted([(feature_names[idx], coef[idx]) for idx in range(len(coef))], key=lambda x: x[1], reverse=True)
    for feature in features[:20]:
      print(feature)
      if feature[0] in word_dict:
        word_dict[feature[0]]+=1
      else:
        word_dict[feature[0]] = 1
  cat_dict[cat]=word_dict



In [ ]:
cat_dict['Mech_Engr']

Classifier to predict accpet and reject

In [80]:
patented_df = df[(df['one_if_small']==1) & (df['Patent_Target']!='pending')]

In [84]:
patented_df['Patent_Target'].value_counts()

0    76829
1    52239
Name: Patent_Target, dtype: int64

In [83]:
patented_df['Patent_Target'] = patented_df['Patent_Target'].apply(lambda x: 1 if x =='abandoned' else 0)

<ipython-input-83-7bf1702e5176>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patented_df['Patent_Target'] = patented_df['Patent_Target'].apply(lambda x: 1 if x =='abandoned' else 0)


In [85]:
patented_df = patented_df[['clean_abstract','Patent_Target']]

In [105]:
patented_df['Patent_Target'].value_counts()

0    76829
1    52239
Name: Patent_Target, dtype: int64

In [102]:
word_dict = {}
for i in range(2):
  only_yes = patented_df.loc[patented_df['Patent_Target']==0,:].sample(52240)
  only_no = patented_df.loc[patented_df['Patent_Target']==1,:]
  all_data = pd.concat([only_yes,only_no],ignore_index=True)
  tfidf_vect = TfidfVectorizer(ngram_range=(1,2))
  train, test= train_test_split(all_data, test_size=0.2, random_state=42)
  Xtrain, ytrain = train['clean_abstract'], train['Patent_Target']
  Xtest, ytest = test['clean_abstract'], test['Patent_Target']
  Xtrain_tfidf = tfidf_vect.fit_transform(Xtrain)
  Xtest_tfidf = tfidf_vect.transform(Xtest)
  lr = LogisticRegression(max_iter=1000)
  vocab = tfidf_vect.vocabulary_
  feature_names = {v: k for k, v in vocab.items()}
  lr.fit(Xtrain_tfidf,ytrain)
  lr.fit(Xtrain_tfidf,ytrain)
  p1=lr.predict(Xtest_tfidf)
  s1=accuracy_score(ytest,p1)
  print("Logistic Regression Accuracy :", "{:.2f}%".format(100*s1))
  coef = lr.coef_[0]
  features = sorted([(feature_names[idx], coef[idx]) for idx in range(len(coef))], key=lambda x: x[1], reverse=True)
  for feature in features[:100]:
    print(feature)
    if feature[0] in word_dict:
      word_dict[feature[0]]+=1
    else:
      word_dict[feature[0]] = 1

Logistic Regression Accuracy : 62.99%
('invention', 3.9130010823789623)
('financial', 2.736853656698412)
('made', 2.5591555485921273)
('business', 2.5273199713182395)
('mean', 2.496146017915021)
('consumer', 2.443356551981419)
('database', 2.2777069493933864)
('comprised', 2.140733260460559)
('offer', 2.0459905757409436)
('social', 2.0002539621012443)
('comprising', 1.9986432740109032)
('owner', 1.9338253089357507)
('adapted', 1.92790982961351)
('power source', 1.9191900124668786)
('fund', 1.9029070707504054)
('providing', 1.8619879704605415)
('portfolio', 1.8463438558595353)
('consists', 1.8306093466649915)
('rating', 1.8179135030269156)
('medical', 1.7902813270614901)
('brain', 1.7869774251152943)
('preferred', 1.7866117998185356)
('would', 1.7800361931945337)
('letter', 1.7471764398945195)
('advertisement', 1.745616202259257)
('amp', 1.7449513447007763)
('quantity', 1.7347282240644566)
('customer', 1.7002476709883458)
('insurance', 1.6980066452007645)
('investment', 1.69747250950749

In [91]:
mech_df = df[(df['one_if_small']==1) & (df['Patent_Target']!='pending') &(df['Parent_Catgeory']=='Mech_Engr')]

In [93]:
mech_df['Patent_Target'] = mech_df['Patent_Target'].apply(lambda x: 1 if x =='abandoned' else 0)

<ipython-input-93-75286fa79878>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mech_df['Patent_Target'] = mech_df['Patent_Target'].apply(lambda x: 1 if x =='abandoned' else 0)


In [94]:
mech_df = mech_df[['clean_abstract','Patent_Target']]

In [96]:
mech_df['Patent_Target'].value_counts(normalize=True)

0    0.566732
1    0.433268
Name: Patent_Target, dtype: float64

In [98]:
word_dict = {}
for i in range(2):
  only_yes = mech_df.loc[patented_df['Patent_Target']==0,:].sample(17500)
  only_no = mech_df.loc[patented_df['Patent_Target']==1,:]
  all_data = pd.concat([only_yes,only_no],ignore_index=True)
  tfidf_vect = TfidfVectorizer(ngram_range=(1,2))
  train, test= train_test_split(all_data, test_size=0.2, random_state=42)
  Xtrain, ytrain = train['clean_abstract'], train['Patent_Target']
  Xtest, ytest = test['clean_abstract'], test['Patent_Target']
  Xtrain_tfidf = tfidf_vect.fit_transform(Xtrain)
  Xtest_tfidf = tfidf_vect.transform(Xtest)
  lr = LogisticRegression(max_iter=1000)
  vocab = tfidf_vect.vocabulary_
  feature_names = {v: k for k, v in vocab.items()}
  lr.fit(Xtrain_tfidf,ytrain)
  lr.fit(Xtrain_tfidf,ytrain)
  p1=lr.predict(Xtest_tfidf)
  s1=accuracy_score(ytest,p1)
  print("Logistic Regression Accuracy :", "{:.2f}%".format(100*s1))
  coef = lr.coef_[0]
  features = sorted([(feature_names[idx], coef[idx]) for idx in range(len(coef))], key=lambda x: x[1], reverse=True)
  for feature in features[:50]:
    print(feature)
    if feature[0] in word_dict:
      word_dict[feature[0]]+=1
    else:
      word_dict[feature[0]] = 1

Logistic Regression Accuracy : 59.99%
('invention', 2.808603648677657)
('comprising', 2.05150361020212)
('nail', 1.7643181656124345)
('staple', 1.6869492925366913)
('pocket', 1.6717357908979233)
('child', 1.6341514714095848)
('mean', 1.6162592325224596)
('hand', 1.5488661002742057)
('word', 1.5185712255843116)
('participant', 1.4907456399839982)
('clothing', 1.463511591002645)
('learning', 1.4049252625397421)
('massage', 1.3957509834734012)
('doll', 1.3709229662960525)
('made', 1.3601876505258483)
('shirt', 1.339807750468816)
('medical', 1.318253107480822)
('razor', 1.3118770414347516)
('sock', 1.3009496924799746)
('providing', 1.290529872256467)
('invention provides', 1.2834206024534942)
('hat', 1.2776069947122568)
('necktie', 1.270204580171348)
('wherein', 1.2685236079519717)
('make', 1.2271900245902834)
('protection', 1.2247949718732523)
('present invention', 1.2230700484090715)
('eyelash', 1.1996869170804891)
('comprised', 1.193302600450676)
('user', 1.1897450667205292)
('body part

In [101]:
df[(df['one_if_small']==1) & (df['Patent_Target']!='pending') &(df['Parent_Catgeory']=='Mech_Engr')]['Gender_Target'].value_counts(normalize=True)

Male      0.834231
Female    0.165769
Name: Gender_Target, dtype: float64